In [ ]:
%%capture

from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyer_loader import JupyerLoader

loader = JupyerLoader()
loader.load("s04_engine_setup")

from converted_notebooks.s04_engine_setup import Engine, ScoredMove, RandomEngine, HumanEngine, playGame
import chess
import random
import IPython.display

# AI Engine

## Basic Evaluation Function

The next engine is the first one
to evaluate chess positions to find better moves. 
Instead of creating the `AbsoluteEvaluationEngine` as one block of code, method and attributes will be added dynamically one after another. This form of dynamic class modification is called monkey patching in the python community 
and is often used as a technique to patch third party code {cite}`plone:glossary`.

In [ ]:
class AbsoluteEvaluationEngine(Engine):
    """Chess engine using the absolute value of the chessboard to evalute moves"""
    pass


A simple way of evaluating a move 
is based on absolute values for each piece.
The dictionary `PIECE_VALUES` associates a chess piece type with a value as commonly defined in {cite}`Capablanca2006` and other books.

In [ ]:
PIECE_VALUES = {
    chess.PAWN: 1,
    chess.KNIGHT: 3,
    chess.BISHOP: 3,
    chess.ROOK: 5,
    chess.QUEEN: 9,
    chess.KING: 0
}

AbsoluteEvaluationEngine.PIECE_VALUES = PIECE_VALUES

The evaluation of the game that is not finished is determined by summing up the piece values of the player to move
and subtracting the piece values of the opponent.
A positive score therefore is good for player to move whereas a negative score is good for the opponent. 
This is implemented in the method `_absolute_heuristic`,
which takes a board as input and returns an integer representing this evaluation. 

In [ ]:
def _absolute_heuristic(self, board: chess.Board) -> int:
    """Calculate the value of all pieces on the board"""

    playerScore = sum(
        self.PIECE_VALUES[board.piece_type_at(square)]
        for square in chess.scan_reversed(board.occupied_co[board.turn])
    )

    opponentScore = sum(
        self.PIECE_VALUES[board.piece_type_at(square)]
        for square in chess.scan_reversed(board.occupied_co[not board.turn])
    )

    return playerScore - opponentScore


AbsoluteEvaluationEngine._absolute_heuristic = _absolute_heuristic

For finished games a method `_evaluate` is definied.
The `chess.Board` function `is_checkmate` can be used to determine if there is a winner. 
For a draw there are multiple functions vor various condiditions like `is_stalemate` or `is_insufficient_material`. 
To make the code more expressive a new function `is_draw` is added to the `chess.Board` class that checks for any of these conditions.

In [ ]:
def is_draw(self) -> bool:
    """Function to check for all any draw condition"""
    return self.is_stalemate() or self.is_insufficient_material(
    ) or self.is_fivefold_repetition() or self.is_seventyfive_moves()


chess.Board.is_draw = is_draw

If the player to move was checkmated, the board is valued as `-100`, which is the lowest possible score. 
In case of a draw the board is valued as `0`. 
If the game is not finished `None` is returned.

In [ ]:
AbsoluteEvaluationEngine.VALUE_CHECKMATE = 100
AbsoluteEvaluationEngine.VALUE_DRAW = 0


def _evaluate(self, board: chess.Board) -> int:
    """Evaluate the current board"""
    if board.is_checkmate():
        return -self.VALUE_CHECKMATE
    if board.is_draw():
        return self.VALUE_DRAW
    return None


AbsoluteEvaluationEngine._evaluate = _evaluate

Both functions `_absolute_heuristic` and `_evaluate` are combined into a single `_evaluate_move` function to score a move. 
It takes a board and a move as a parameter and returns a scored move from the perspective of the White player. 

In [ ]:
AbsoluteEvaluationEngine.PLAYER_MULTIPLIER = {
    chess.WHITE: 1, chess.BLACK: -1
}


def _evaluate_move(self, board: chess.Board, move: chess.Move) -> ScoredMove:
    """Evaluate a single move from white perspective"""

    board.push(move)
    score = self._evaluate(board)
    if score is None:
        score = self._absolute_heuristic(board)
    score *= self.PLAYER_MULTIPLIER[board.turn]
    board.pop()

    return ScoredMove(score, move)


AbsoluteEvaluationEngine._evaluate_move = _evaluate_move

Next, the `analyse` method can be defined to score all legal moves using the `_evaluate_move` method.

The scored moves are then shuffled and, afterwards, sorted by their score, 
creating a different order of moves having the same score, 
depending on the state of the RNG.

By default, the python `sort` method sorts from lowest to highest. 
Therefore, the first move is the best for Black, unless the sorting order is reversed.

TODO: Explain _evaluate_moves

In [ ]:
def _evaluate_moves(self, board: chess.Board):
    return [self._evaluate_move(board, move) for move in board.legal_moves]


AbsoluteEvaluationEngine._evaluate_moves = _evaluate_moves


def analyse(self, board: chess.Board) -> list[ScoredMove]:
    """Analyse method using _absolute_heuristic"""
    # TODO: Explain use of copy here
    nextMoves = self._evaluate_moves(board.copy(stack=False))
    random.shuffle(nextMoves)

    whitesTurn = board.turn is chess.WHITE
    nextMoves.sort(reverse=whitesTurn)

    return nextMoves


AbsoluteEvaluationEngine.analyse = analyse

The `play` method will simply return the best move from `analyse` as an `chess.engine.PlayResult`. 
In case of having multiple best values, the taken move is random as the `analyse` shuffles the list.

In [ ]:
def play(self, board: chess.Board) -> chess.engine.PlayResult:
    bestScoredMove = self.analyse(board)[0]
    return chess.engine.PlayResult(move=bestScoredMove.move, ponder=None)


AbsoluteEvaluationEngine.play = play

The `AbsoluteEnhancedRandomEngine` strategy is to capture everything the engine can. 
Additionally, it is also able to find mate in one. 
Therefore, it has very good chances to win against the `RandomEngine`.

In [ ]:
random.seed(42)
board = chess.Board()
playGame(board, AbsoluteEvaluationEngine(), RandomEngine(), displayBoard=False)
IPython.display.display(board)
print(board.outcome())

To verfify the correctness the `_absolute_heuristic` its output is checked against some famous chess games.

In [ ]:
random.seed(42)
engine = AbsoluteEvaluationEngine()

# TODO: Rewrite for _evaluate_move

# Topalov, Veselin (2740) vs. Shirov, Alexei (2710)
SHIROV_SACRIFICE = "8/8/4kpp1/3p1b2/p6P/2B5/6P1/6K1 b - - 0 47"
board = chess.Board(SHIROV_SACRIFICE)
#IPython.display.display(board)
score = engine._evaluate(board)
# Black has 2 points more and White moved last, so the score should be -2
# assert score == -2, f"{score} != -2"

# Evgeny Yuryevich Vladimirov vs. Vladimir Viktorovich Epishin
VLADIMIROV_THUNDERBOLT = "r4k1r/1b2bPR1/p4n2/3p4/4P2P/1q2B2B/PpP5/1K4R1 w - - 0 26"
board = chess.Board(VLADIMIROV_THUNDERBOLT)
# IPython.display.display(board)
# Black has 10 points more and Black moved last, so the score should be 10
score = engine._evaluate(board)
# assert score == 10, f"{score} != 10"

Additionally, a special chess position was constructed to test the `analyse` method. In this position, different moves such as capturings and promotions are possible.

In [ ]:
random.seed(42)
PROMOTION_POSITION = "Kn2rn1k/1p2P3/8/8/8/8/8/8 w - - 0 1"
board = chess.Board(PROMOTION_POSITION)
IPython.display.display(board)
# Black has 11 points more and Black moved last, so the score should be 11
score = engine._evaluate(board)
# assert score == 11, f"{score} != 11"
# Analyse scores are from the persective of the White Player!
result = engine.analyse(board)
assert result == [ScoredMove(0, chess.Move.from_uci("e7f8q")),
                  ScoredMove(-4, chess.Move.from_uci("e7f8r")),
                  ScoredMove(-6, chess.Move.from_uci("e7f8b")),
                  ScoredMove(-6, chess.Move.from_uci("e7f8n")),
                  ScoredMove(-10, chess.Move.from_uci("a8b7")),
                  ScoredMove(-11, chess.Move.from_uci("a8b7"))], f'{result} is not correct'

## Minimax Algorithm

Until now, a move was evaluated according to the value the heuristic gives to the resulting position. Thus the chess engine lacks the foresight to find tactical combinations or a mate in several moves.
With the help of a search algorithm all future positions can be found, which are created after playing a certain number of moves. {numref}`game-tree-figure` shows how the resulting positions in the game `TicTacToe` can be represented as a search tree. The root node is the initial position, while the leaf nodes contain the final positions that will be evaluated. A special feature of TicTacToe is that the leaf nodes are terminal states. In chess, on the other hand, the search usually has to be stopped at a certain depth, because the number of states is simply too large even for modern computers. The leaf nodes are therefore not necessarily terminal states and are thus evaluated with the already known heuristics. 

```{figure} ./images/tictactoe-gametree.jpg
---
height: 150px
name: game-tree-figure
---
Sample TicTacToe search tree
```

TODO: This cite belongs to the figure
{cite}`TicTacToe_Search_Tree`

By limiting the search depth, however, the original problem is only partially solved. As before, it can happen that the considered final position ends, for example, in the middle of a slugfest and is correspondingly wrongly estimated. It also often leads to the fact that the engine tries to delay a foreseeable problem by giving chess etc. until it is no longer visible due to the search depth. 
This is called the horizon effect, which can have a very strong impact on the engine's playing strength and is discussed accordingly in later chapters.
{cite}`Horizon_Effect`

First, however, some search algorithms are presented, starting with the `MiniMax algorithm`. For this purpose, first a class `MiniMaxEngine` is defined, which inherits from `AbsoluteEvaluationEngine`, since for example the heuristic is reused. The constructor of the class initializes the attribute `look_ahead_depth`, which specifies the depth of the search tree.

In [ ]:
class MiniMaxEngine(AbsoluteEvaluationEngine):
    """Chess engine looking a fixed number of moves ahead using the minimax algorithm"""

    def __init__(self, look_ahead_depth):
        self.look_ahead_depth = look_ahead_depth

The actual `MiniMax algorithm` is realized recursively with the function `_value`. This function creates the search tree and takes the board and the remaining depth of the current node of the search tree as parameters. In each recursive pass, the method returns the value of the current node from the perspective of white. 

First, the method `_evaluate` checks whether the current position is a terminal state. If this is the case, the evaluation of this from the perspective of white is returned accordingly. 
Otherwise, it is checked whether the maximum search depth has been reached and it is therefore a leaf node. In this case the heuristic method `_absolute_heuristic` is used and the value from the perspective of white is returned.

If neither of the termination conditions is true, the recursive part of the method is executed. For each possible move the new position is generated and recursively calculated with `_value`. In total, a list of values is generated for each child node in the search tree. 

Finally, the question remains how the node itself is evaluated based on the evaluations of the child nodes. The key idea of the `MiniMax algorithm` is that there is a minimizing and a maximizing player. When it is White's turn, he tries to maximize the valuation for his move. Accordingly, in this case the maximum value of the child nodes is returned. If, on the other hand, it is black's turn, he tries to minimize the valuation for his move. Therefore, the minimum value of the child nodes is returned. 
Overall, the algorithm assumes that both players want to win and play the best move for themselves. The respective position is thus evaluated as the future position that would be reached if both players played optimally.

In [ ]:
def _value(self, board: chess.Board, depth: int) -> int:
    if score := self._evaluate(board):
        return self.PLAYER_MULTIPLIER[board.turn] * score
    if depth == 0:
        return self.PLAYER_MULTIPLIER[board.turn
                                      ] * self._absolute_heuristic(board)

    scores = []
    for move in board.legal_moves:
        board.push(move)
        scores.append(self._value(board, depth - 1))
        board.pop()

    if board.turn is chess.WHITE:
        return max(scores)
    else:
        return min(scores)


MiniMaxEngine._value = _value

Finally, only the `_evaluate_move` must be adjusted so that it calculates the value of the move using the `_value` function just defined. The function is initially called with the maximum search depth.

In [ ]:
def _evaluate_move(self, board: chess.Board, move: chess.Move):
    board.push(move)
    score = self._value(board, self.look_ahead_depth)
    board.pop()
    return ScoredMove(score=score, move=move)


MiniMaxEngine._evaluate_move = _evaluate_move

To demonstrate how the algorithm works, a special chess position is constructed in which only a limited number of moves are possible and the evaluation varies greatly depending on the move.

In [ ]:
sample_minimax_board = chess.Board("7k/7P/7P/8/8/p7/8/7K b - - 0 1")
IPython.display.display(sample_minimax_board)

An additional auxiliary class `MinMaxTree` was written, which however only serves to visualize the search tree and is therefore not explained in more detail. An instance of the class can be created with the method `add_tree_to_engine`. If then a position is analyzed with `analyze`, the tree can be drawn afterwards with `draw`. 

In [ ]:
import MinMaxTree

random.seed(42)

engine = MiniMaxEngine(look_ahead_depth=2)
tree = MinMaxTree.add_tree_to_engine(engine)
print(engine.analyse(sample_minimax_board))
tree.draw()

In the constructed example, the search tree can still be well surveyed. This is due in particular to the low branching factor in the first two moves. On average, however, the branching factor is 35 - 38 {cite}`Branching_Factor`. As a consequence, the number of nodes with higher depth quickly exceeds the computational capacity of modern computers. 

The following position, which arises from the Exchange Variation of the Declined Queen's Gambit, will demonstrate this.

In [ ]:
middlegame_board = chess.Board(
    "r1bqrnk1/pp2bppp/2p2n2/3p2B1/3P4/2NBPN2/PPQ2PPP/R4RK1 w - - 7 11"
)

In [ ]:
IPython.display.display(middlegame_board)
print(f"Number of moves: {len(list(middlegame_board.legal_moves))}")

If this position is evaluated with the `MiniMaxEngine` at depth two, it is immediately noticeable that the calculation is considerably slower. The evaluation of the tree also shows that it contains a total of over 70'000 nodes. At a depth of three, the number of nodes increases to 2'342'944. 

In [ ]:
random.seed(42)

engine = MiniMaxEngine(look_ahead_depth=2)
tree = MinMaxTree.add_tree_to_engine(engine)
result_minimax = engine.analyse(middlegame_board)
print(result_minimax)
tree.count()

## AlphaBeta Pruning Algorithm

The previous example makes clear that the primary goal to increase the search depth must be a reduction of the nodes. The `MiniMax algorithm` is a depth-first search where the paths have to be evaluated completely one after the other. If it is the maximizing player's turn, i.e. white, the first evaluation of the path gives the minimum value white can reach. Each further path must exceed this value so that it is ultimately played by white. Similarly, for black on the move, each further path must undercut the previous value for it to be played by black. 

The `AlphaBeta Pruning Algorithm` builds on this idea and introduces two additional parameters `alpha` and `beta` for the `_value` function. `alpha` is the value that the current position has at least and beta is the value that the position has at most. Therefore `alpha <= _value(board, depth, alpha, beta) <= beta` holds.

First, a new class `AlphaBetaEngine` is defined again, which inherits from `MiniMaxEngine`.

In [ ]:
class AlphaBetaEngine(MiniMaxEngine):
    """Chess engine looking a fixed number of moves ahead using the alpha beta pruning algorithm"""

The `_value` function has the same interface as before except for the two new parameters `alpha` and `beta`. The termination conditions of the recursive function are also the same. 

For the maximizing player, hence the white player, `alpha` stores the current best score that can be achieved in that position. For each child node, alpha is therefore incremented with the statement `alpha = max(alpha, value)` if its value is higher. However, if `alpha` or the score of a child node is greater than `beta`, the search can be terminated prematurely. This is called a `beta cutoff`. The reason for this is that with `beta` Black could already achieve a better score for himself in another position and thus will not let White get into the current situation at all. Thus it is of no use to White to analyze the further moves. 

The same applies to the minimizing player. His currently best score is stored in `beta` and always lowered with the instruction `beta = min(beta, value)` if a child node has a low and therefore better value for black. If `beta` or the value of the child node is below `alpha`, the search can be aborted. This is called an `alpha cutoff`. Here, too, White had already found a better move for himself one level higher with `alpha` and will therefore not let Black get into this position. 

In [ ]:
def _value(self, board: chess.Board, depth: int, alpha: int, beta: int) -> int:
    if score := self._evaluate(board):
        return self.PLAYER_MULTIPLIER[board.turn] * score
    if depth == 0:
        return self.PLAYER_MULTIPLIER[board.turn
                                      ] * self._absolute_heuristic(board)

    if board.turn is chess.WHITE:
        for move in board.legal_moves:
            board.push(move)
            value = self._value(board, depth - 1, alpha, beta)
            board.pop()
            if value >= beta:
                return value
            alpha = max(alpha, value)

        return alpha
    else:
        for move in board.legal_moves:
            board.push(move)
            value = self._value(board, depth - 1, alpha, beta)
            board.pop()
            if alpha >= value:
                return value
            beta = min(beta, value)

        return beta


AlphaBetaEngine._value = _value

Finally, the `_evaluate_move` method must be adapted. This is identical to the implementation of the `MiniMaxEngine` with exception of the `_value` call. There you have to pass an initial value for `alpha` and `beta`. Since no move has been played yet, White can in any case achieve the worst possible result, he is set to mate. Accordingly `alpha` is initialized with `-self.VALUE_CHECKMATE`. The best possible result for white is to set mate himself, accordingly `beta` is initialized with `self.VALUE_CHECKMATE.

In [ ]:
def _evaluate_move(self, board: chess.Board, move: chess.Move):
    board.push(move)
    score = self._value(
        board,
        self.look_ahead_depth,
        -self.VALUE_CHECKMATE,
        self.VALUE_CHECKMATE
    )
    board.pop()
    return ScoredMove(score=score, move=move)


AlphaBetaEngine._evaluate_move = _evaluate_move

Now the `AlphaBetaEngine` can evaluate the same constructed position `sample_minimax_board`. You can see in the graph that some nodes are marked with a question mark. These are the nodes that in this case did not have to be evaluated, because the result of the parent node was already defined before due to alpha and beta cutoffs.

In [ ]:
random.seed(42)

engine = AlphaBetaEngine(look_ahead_depth=2)
# engine = AlphaBetaEngine(look_ahead_depth=3)
tree = MinMaxTree.add_tree_to_engine(engine)
print(engine.analyse(sample_minimax_board))
tree.draw()

For the second position 'middlegame_board', the number of nodes can again be calculated for depths two and three. At a depth of two, with 7'459 only about 10% of the previous nodes have to be examined, 62'660 paths were pruned. At a depth of three, with 10,7628 nodes only about 5% of the previous nodes have to be evaluated.

In [ ]:
random.seed(42)

engine = AlphaBetaEngine(look_ahead_depth=2)
# engine = AlphaBetaEngine(look_ahead_depth=3)
tree = MinMaxTree.add_tree_to_engine(engine)
result_alphabeta = engine.analyse(middlegame_board)
tree.count()

From the operation of the algorithm, it is obvious that it is advantageous if the best path is evaluated first. In the best case, it is possible that the `x` nodes considered at `MiniMax` decrease to `sqrt(x)` when `Alpha Beta Pruning` is applied. If, on the other hand, the moves are sorted the other way around, so that the worst path is started with, no pruning is possible at all. {cite}`Alpha_Beta`

## NegaMax Algorithmus 

The `NegaMax algorithm` does not lead to a direct improvement of the engine, but simplifies the code of the `MiniMax algorithm` and all algorithms based on it. The basic idea here is that the evaluation of each node is done from the perspective of the player on the move. Thus both players are maximizing and the previous case distinction is no longer necessary. 

For the previous implementation of `_value` for the `AlphaBeta Pruning Algorithm`, this first of all changes the return values of the termination conditions. Since the value is now from the perspective of the player on the move and the functions `_evaluate` and `_absolute_heuristic` also evaluate from the perspective of the player on the move, the value can be returned directly. 
Furthermore, the case distinction is completely omitted and only the code for the maximizing player is needed. There the recursive call of the `_value` function changes to `value = -1 * self._value(board, depth - 1, -beta, -alpha)`. The return value, as well as alpha and beta, are negated to be from the player's perspective on the move. Additionally, alpha and beta must be swapped. 

In [ ]:
def _value(self, board: chess.Board, depth: int, alpha: int, beta: int) -> int:
    if score := self._evaluate(board):
        return score
    if depth == 0:
        return self._absolute_heuristic(board)

    for move in board.legal_moves:
        board.push(move)
        value = -1 * self._value(board, depth - 1, -beta, -alpha)
        board.pop()
        if value >= beta:
            return value
        alpha = max(alpha, value)

    return alpha


AlphaBetaEngine._value = _value

The `_evaluate_move` needs to be adjusted so that its returned result is again from the perspective of white as before.

In [ ]:
def _evaluate_move(self, board: chess.Board, move: chess.Move):
    board.push(move)
    score = self._value(
        board,
        self.look_ahead_depth,
        -self.VALUE_CHECKMATE,
        self.VALUE_CHECKMATE
    )
    score *= self.PLAYER_MULTIPLIER[board.turn]
    board.pop()
    return ScoredMove(score=score, move=move)


AlphaBetaEngine._evaluate_move = _evaluate_move

Now the `middlegame_board` position can also be analyzed by the `NegaMax` variant and afterwards it can be verified that all three implementations actually evaluate the position in the same way.

In [ ]:
random.seed(42)

engine = AlphaBetaEngine(look_ahead_depth=2)
tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
result_negamax = engine.analyse(middlegame_board)
tree.count()

assert result_minimax == result_alphabeta == result_negamax

## Iterative Deepening

The algorithms shown so far have the advantage of a low memory consumption, as usual in a depth-first searches. Thus, at most the current path must always be kept in memory. A disadvantage is, however, that it is difficult to build in a time break. If the search is simply aborted after a certain time, some paths have been evaluated completely, while other paths have not been considered at all. Another problem currently with the `AlphaBeta Pruning Algorithm` is that in the optimal case, good moves should be considered first, in order to be able to prune as many paths as possible. So far, however, there is no heuristic to perform this sorting of the moves.

The idea of iterative depth-first search provides a possible solution to both problems. In contrast to the normal depth-first search, the depth is iteratively increased until the maximum depth is reached. The obvious disadvantage of this is more overhead, since nodes that have already been analyzed are looked at again in the next iterative pass. However, as seen in previous examples, the number of new nodes increases so dramatically for each increase in search depth that re-analyzing previous nodes is negligible.

The advantage of iterative depth-first search, however, is that the results obtained from previous runs with lower search depths can be used. On the one hand, these results can be returned if the time limit expires in the next run. On the other hand, based on the previous run, the sorting of moves for the next one can be done. 

At first only the iterative deepening algorithmn itself shall be implemented in a new class `IterativeAlphaBeta`, which inherits from `AlphaBetaEngine`.

In [ ]:
class IterativeAlphaBeta(AlphaBetaEngine):
    """Chess engine looking a fixed number of moves ahead using the alpha beta pruning algorithm"""

    def __init__(self, max_look_ahead_depth):
        self.max_look_ahead_depth = max_look_ahead_depth

The `_evaluate_move` method must be adapted so that the depth is now a parameter.

In [ ]:
def _evaluate_move(self, board: chess.Board, move: chess.Move, depth: int):
    board.push(move)
    score = self._value(
        board, depth, -self.VALUE_CHECKMATE, self.VALUE_CHECKMATE
    )
    score *= self.PLAYER_MULTIPLIER[board.turn]
    board.pop()
    return ScoredMove(score=score, move=move)


IterativeAlphaBeta._evaluate_move = _evaluate_move

In the `_evaluate_moves` method, a simple loop can now be built in, which increases the depth step by step until the desired maximum depth is reached.

In [ ]:
def _evaluate_moves(self, board: chess.Board):
    print(f"Max depth: {self.max_look_ahead_depth}")

    for depth in range(self.max_look_ahead_depth + 1):
        scoredMoves = [
            self._evaluate_move(board, move, depth)
            for move in board.legal_moves
        ]

        print(f"Depth {depth}")
        # print(f"result: {scoredMoves}\n")

    return scoredMoves


IterativeAlphaBeta._evaluate_moves = _evaluate_moves

Again, it can be verified that the new implementation still evaluates the already known position in the same way.

In [ ]:
random.seed(42)

engine = IterativeAlphaBeta(max_look_ahead_depth=2)
result_iterativeAlphaBeta = engine.analyse(middlegame_board)

assert result_iterativeAlphaBeta == result_negamax

## Transpositionstabellen

In the last section of this chapter we will look at transposition tables. Essentially, they are a cache for the search tree that serves several purposes. The first use case and eponym are transpositions. In chess one speaks of a transposition, if the same position is obtained by different move sequences. In this case, with the help of a cache, the position has to be evaluated only once. In some algorithms, such as the 'MDTF algorithm', a node of the search tree is also evaluated several times in succession. In this case it is therefore possible to fall back on the already calculated results in the cache. Finally, it is also possible, e.g. in an iterative deepening framework, to perform move ordering after a search pass through the cache. A possible implementation of the `principal variation search`, for example, uses the cache to determine the principal variation of the previous search pass.

First, we have to consider how an entry in the cache is structured. Besides the value of a node, the depth for which this value is valid is of course important. Furthermore, it could be seen with the `AlphaBeta algorithm` that it performs optimizations based on the `alpha` and `beta` value and thus on previously considered other paths in the tree. Thus, the value is not necessarily valid in a transposition that was reached by other means. One possibility would therefore be to store `alpha` and `beta` as well. 

A more meaningful way, however, is to classify the different nodes within a search tree:

1. `alpha < score < beta`: All child nodes have been examined and the value is always exact, even for transpositions. This is often referred to as `PV-Node`.
1. `alpha < beta <= score`: The search was aborted by a beta cut, therefore it is also called a `Cut-Node`. The score here is a lower limit, the actual value could be larger. 
1. `score <= alpha < beta`: Here all child nodes were examined too, however beta cutoffs occured. These nodes are also referred to as `All-Node`. Here the value acts as an upper bound, the actual value could be lower. 

In general it can be said that at least the root node and the leftmost node must be `PV-Nodes`, because `alpha` and `beta` cannot cause cuttofs there yet. The children of `PV-Nodes` can in turn also be `PV-Nodes` or `Cut-Nodes` if a beta cuttoff has occurred. After `Cut-Nodes` follow in each case alternating `All Nodes` and again `Cut-Nodes`. cite:[Node_Types].

The node type is stored in an enum. As designation the effect of the value was chosen, therefore, whether this is exact, a lower limit or an upper limit.

In [ ]:
from enum import Enum


class NodeType(Enum):
    EXACT = 0  # PV Node
    LOWER_BOUND = 1  # Cut Node
    UPPER_BOUND = 2  # All Node

Based on this a dedicated data class is created
to store the node type, value and depth of a position.

In [ ]:
from dataclasses import dataclass


@dataclass
class CachedPositionEntry:
    type: NodeType
    value: int
    depth: int

A dedicated class `AlphaBetaCache` is created
to provide an abstraction for storing and loading entries.
The entries are interally stored in a `dict` 
and wrapped methods are provided to reset the cache
or get its size.

In [ ]:
class AlphaBetaCache:

    def __init__(self):
        self.cache = dict()

    def clear(self):
        self.cache.clear()

    def size(self):
        return len(self.cache)

For a cache entry their needs to be a key to identify the position.
This key is calculated using the method `get_key`,
which takes a board as input and returns a key representing the position in a unique way.
As this function is called wether there is a matching cached entry or not
it needs to be fast.
A popular way of identifying positions in chess engines is the usage of Zobrist Hashes,
as they can be determined incrementally cite:[Zobrist_Hashing].
As the `python-chess` library already has an internal method `_transposition_key`
this is used.
It is rather quick
as only internal representations of the position are used to calculate it
and the calculation is already done on every `push` and `pop`.
To use Zobrist Hashes in a useful way
the `python-chess` library would need to be patched.

In [ ]:
def get_key(self, board: chess.Board):
    return board._transposition_key()


AlphaBetaCache.get_key = get_key

To save an entry to the cache its node type needs to be calculated.
This is done by the auxilary function `_get_node_type`
which takes a value, the depth, alpha and beta as parameters and returns the node type.
It implements the typing as described above 
and additionally recognizes the leaf nodes as PV Nodes by their depth of 0.

In [ ]:
def _get_node_type(self, value: int, depth: int, alpha: int, beta: int):
    # TODO: Correct in some cases, but not all (for instance with quiescene search, which does cutoffs)
    # if depth == 0:
    #     return NodeType.EXACT
    if value <= alpha:
        return NodeType.UPPER_BOUND
    if value >= beta:
        return NodeType.LOWER_BOUND
    return NodeType.EXACT


AlphaBetaCache._get_node_type = _get_node_type

The `store_cache` method
adds an entry 
defined by the key returned by the `get_key` method
to the cache.
The entry consists of the node type as returned by the `_get_node_type` method,
the value and the current depth.
The parameters are therefore the parameters of `_get_node_type` (value, alpha, beta) and the key.

Initially the cache is checked for an already existing entry.
If there is a matching entry and its depth is deeper than the current depth, 
the current value is not stored in the cache 
as the other value has been calculated using more ressources already.
This is called depth first replacement
and is a popular way to handle replacing cache values,
but it is not the only way to do so. 
If there is no such entry,
the node type is evaluated and the tuple as described above is added to the cache.

In [ ]:
def store_cache(self, key: int, value: int, depth: int, alpha: int, beta: int):
    # depth first replacement
    entry = self.cache.get(key)
    if entry and entry.depth >= depth:
        return

    node_type = self._get_node_type(value, depth, alpha, beta)
    self.cache[key] = CachedPositionEntry(node_type, value, depth)


AlphaBetaCache.store_cache = store_cache

The `load_cache` method returns an entry and it's type from the cache given a key.
Additionaly the depth is supplied as a parameter
as the cache entry is only relevant 
if it has the same or a greater depth.
If there is no matching entry
or the depth of the cached value is too shallow
a `KeyError` exception is thrown.

In [ ]:
def load_cache(self, key: int, depth: int):
    entry = self.cache[key]
    if entry.depth < depth:
        raise KeyError

    return (entry.type, entry.value)


AlphaBetaCache.load_cache = load_cache

The class `AlphaBetaCache` is fully implemented,
but to use it the `_value` needs to be aware of it.
Therefore a decorator `cache_alpha_beta` is created
to check the cache for a matching entry
and to store the new result
if there was no matching entry yet.

The decorator calculates the key for the current position 
and tries to get a matching entry from the cache 
via `load_cache`.
If there is such an entry a differentiation needs to be made;
an exact value can be returned immediately
while a lower bound may increase `alpha`
and an upper bound may decrease `beta`.
This might lead to a cutoff
in case the new `alpha` is at least `beta`
and the cached value being returned immediately.

If the value is not returned immediately
or there was no value in the cache at all
the `value_function` is called
and the returned value is stored in the cache via `store_cache`
before returning the result.

In [ ]:
from functools import wraps


def cache_alpha_beta(value_function):

    @wraps(value_function)
    def cached_value(
        self, board: chess.Board, depth: int, alpha: int, beta: int
    ):
        cacheKey = self.cache.get_key(board)
        try:
            type, value = self.cache.load_cache(cacheKey, depth)

            if type == NodeType.EXACT:
                return value
            elif type == NodeType.LOWER_BOUND:
                alpha = max(value, alpha)
            else:  # type == NodeType.UPPER_BOUND
                beta = min(value, beta)

            if alpha >= beta:
                return value
        except:
            pass

        value = value_function(self, board, depth, alpha, beta)
        self.cache.store_cache(cacheKey, value, depth, alpha, beta)
        return value

    return cached_value

To use the cache a dedicated class `IterativeAlphaBetaCached` is created inheriting from `IterativeAlphaBeta`.
On construction of an object of the `IterativeAlphaBetaCached` class
the cache is created
and each call of `_evaluate_moves` empties the cache
to prevent stale data influencing the analysis.
To really use the cache
the `_value` function gets the newly created `cache_alpha_beta` decorator.

In [ ]:
class IterativeAlphaBetaCached(IterativeAlphaBeta):

    def __init__(self, max_look_ahead_depth: int):
        super().__init__(max_look_ahead_depth)
        self.cache = AlphaBetaCache()

    def _evaluate_moves(self, board: chess.Board):
        self.cache.clear()
        return super()._evaluate_moves(board)

    @cache_alpha_beta
    def _value(
        self, board: chess.Board, depth: int, alpha: int, beta: int
    ) -> int:
        return super()._value(board, depth, alpha, beta)

As the cache usage should only influence the performance
the result of the caching engine should match that of the non-caching engine.
This is checked here as seen in previous examples.

In [ ]:
random.seed(42)

engine = IterativeAlphaBetaCached(max_look_ahead_depth=2)
tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
result_iterativeAlphaBetaCached = engine.analyse(middlegame_board)

assert result_iterativeAlphaBetaCached == result_iterativeAlphaBeta